## **Transfering ES model validated parameters to data-scarce watersheds**

publication: "Using machine learning for long-term calibration and validation of water quality ecosystem service models in data-scarce regions"

author: "Valladares-Castellanos M, de Jesús Crespo R, Douthat T"


github: https://github.com/LSU-EPG/Puerto-Rico-ES-Project/tree/main/Data_Scarcity_Framework

**Summary**: The following notebook describes the last step of the methodological framework to transfered the calibrated and validated parameters to non-reference watershed to quantify and map ES for the entire study area. In the notebook we used as an example the suitable values found for the case study watersheds: cluster 1 TFA200 and Kb 0.5, and cluster 2 TFA 50 and Kb 4.0.

####  1. Install the Packages

In [ ]:
## The necessary packages 
import logging
import sys
import natcap.invest.ndr.ndr
import natcap.invest.utils
import pandas

#### 2. Start ES Model Batch Prediction

##### 2.1 Load the validated parameters from a CSV table

In [ ]:
param_table_path = 'path/Param_BestTFA_Kb.csv' 
param_df = pd.read_csv(param_table_path)
print(param_df)

##### 2.2 Apply batch model prediction

In [ ]:
## set up the parameters 
LOGGER = logging.getLogger(__name__)
root_logger = logging.getLogger()

handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(
    fmt=natcap.invest.utils.LOG_FMT,
    datefmt='%m/%d/%Y %H:%M:%S ')
handler.setFormatter(formatter)
logging.basicConfig(level=logging.INFO, handlers=[handler])

# Set the parameters folder location
args = {
    'biophysical_table_path': 'path/biophysical_table.csv',
    'calc_n': True,  
    'calc_p': True,
    'dem_path': 'path/DEM.tif',
    'subsurface_critical_length_n': 'value',
    'subsurface_eff_n': 'value',
    'workspace_dir':'workspace_directory_path',
}

# # Loops through the values 
for year in param_df['year'].unique():
    for cluster in param_df['watershed_cluster'].unique():
        # Filter the table for the specific year and cluster combination
        filtered_params = param_df[(param_df['year'] == year) & (param_df['watershed_cluster'] == cluster)]

        if not filtered_params.empty:
            # Assuming the table has one row for each year and cluster combination
            tfa = filtered_params['TFA'].values[0]  # Get the TFA value
            kb = filtered_params['Kb'].values[0]  # Get the Kb value

            # Set the TFA and Kb parameters
            args['threshold_flow_accumulation'] = tfa
            args['k_param'] = kb

            # Set the paths for LULC and runoff proxy based on the year
            lulc = f'path/LULC_{year}.tif'
            runoff = f'path/Q_mm_{year}_.tif'
            args['lulc_path'] = lulc
            args['runoff_proxy_path'] = runoff

            # Set the HUC12 watershed path based on the cluster
            study_area_watershed = f'path/Watershed_Cluster{cluster}.shp'
            args['watersheds_path'] = study_area_watershed

            # Define a suffix for the results to include year, cluster, TFA, and Kb values
            args['results_suffix'] = f'Nut_year{year}_cluster{cluster}_TFA{tfa}_kb{kb}'

            # Run the model
            if __name__ == '__main__':
                natcap.invest.ndr.ndr.execute(args)
        else:
            print(f"Warning: No calibration parameters found for year {year}, cluster {cluster}. Skipping.")

#### 3. References:

Campanhão, L. M. B., & Ranieri, V. E. L. (2023). Influence of forest proportion and configuration at the watershed and riparian zone scales on sediment yield: a simulation experiment. Landscape Ecology, 38(11), 2839-2860. https://doi.org/10.1007/s10980-023-01751-6

Natural Capital Project. (2023). InVEST 3.14.0. In Stanford University, University of Minnesota, Chinese Academy of Sciences, The Nature Conservancy, World Wildlife Fund, Stockholm Resilience Centre and the Royal Swedish Academy of Sciences. https://naturalcapitalproject.stanford.edu/software/invest

Valladares-Castellanos, M., de Jesús Crespo, R., Xu, Y. J., & Douthat, T. H. (2024). A framework for validating watershed ecosystem service models in the United States using long-term water quality data: Applications with the InVEST Nutrient Delivery (NDR) model in Puerto Rico. Science of the Total Environment, 949, 175111. https://doi.org/https://doi.org/10.1016/j.scitotenv.2024.175111 